In [8]:
import sys
sys.path.append('/home/sensei/jupy-notebooks/Analytics/PorterFarms/')
print("============================================")
print("/  AllTroughs is running.                  /")
print("============================================")
from datetime import datetime, timedelta
import pytz
import json
import copy
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import configparser

config = configparser.ConfigParser()
config.read("../../../analytics_secrets.ini")

_ACTIVE_STANDBY = config['DEFAULT']['role']
if _ACTIVE_STANDBY == 'STANDBY':
    print("STANDBY")
    raise SystemExit("Stop right there!")
else:
  _SLACK_TOKEN = config['slack']['token']
  _CHIRPSTACK_USER = config['chirpstack']['user']
  _CHIRPSTACK_PASS = config['chirpstack']['password']
  _DB_HOST  = config['kanjidb']['dbhost']
  _DB_PORT  = config['kanjidb']['dbport']
  _DB_NAME  = config['kanjidb']['dbname']
  _DB_USER  = config['kanjidb']['dbuser']
  _DB_PASS  = config['kanjidb']['dbpass']
    
  _USE_DROPBOX   = config['dropbox']['usedropbox']
  _FALLBACK_IMAGE = config['dropbox']['fallbackimage']  
    
  _UTC_OFFSET = int(config['DEFAULT']['utcoffset'])

  _THRESHOLD_WMA = float(config['analytics']['waterLevelMin'])
  _THRESHOLD_WMAMEAN = float(config['analytics']['thresholdMean'])
  _MIN_SD = float(config['analytics']['minSd'])
  _MIN_MEAN = float(config['analytics']['minMean'])
  _MAX_MEAN = float(config['analytics']['maxMean'])

  _LOG_DEBUG = 0
  _LOG_INFO  = 1
  _LOG_ERROR = 2
  _LOG_LEVEL = int(config['DEFAULT']['loglevel'])

def logger(level, message):
    if level >= _LOG_LEVEL:
      print(message)

logger(_LOG_DEBUG, "{} {} {} {} {}".format(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS))

import kanjiticketing as kt

conn = kt.getKanjiDbConnection(_DB_HOST, _DB_PORT, _DB_NAME, _DB_USER, _DB_PASS)
if conn is not None:
  print("Welcome to Jupyter Notebook.  You are connected to the Kanji database!")
else:
  print("You are not connected to the database.")

/  AllTroughs is running.                  /
Welcome to Jupyter Notebook.  You are connected to the Kanji database!


**Strategy**
Determine the most likely condition at the mote:

The sensor is WET, DRY, or BAD

**FALSE POSITIVES: (WET INDICATION)**

If the sensor emitter fails to produce IR or if the sensor phototransistor fails OPEN

**FALSE NEGATIVES: (DRY INDICATION)**

If the sensor phototransistor fails SHORT

The secret recipe to protect against false indications consists of two things. 1) that we cycle the IR emitter at 50% duty factor. In each half-cycle the voltage at the phototransistor collector is sampled.  This process is repeated for a number of cycles (e.g. 10).  Then the mean and standard deviation are calculated.  It is the standard deviation which helps us to establish that the sensor is operating properly. 2) attached near the face of the emitter is an optically reflective surface.  The purpose of this surface is to reflect some of the IR that escapes from the sensor when it is immersed in fluid.


In [10]:
#_LOG_LEVEL = _LOG_DEBUG
_PREFERRED_IMAGE =  "https://www.dropbox.com/s/0ejb4hu35n9sbdd/low-water.jpg?raw=1"            

if _USE_DROPBOX == 'true':
  locationimageurl = _PREFERRED_IMAGE
else:
  locationimageurl = _FALLBACK_IMAGE    

_CATTLE_TROUGH_MONITOR = 10004
_AVERAGING_INTERVAL_MINUTES = 20
_AGE_THRESHOLD_SECONDS = 900

#Ticket Types
LOW_WATER_LEVEL = 10001
BAD_FLUIDLEVEL_SENSOR = 10005

now = datetime.now(pytz.utc)  #tz Aware
starttime = now - timedelta(hours=0, minutes=_AVERAGING_INTERVAL_MINUTES)
logger(_LOG_DEBUG, "Current time is {}".format(now))
logger(_LOG_DEBUG, "Query timestamp will start at {}".format(starttime))

nodequery = "SELECT * FROM kanji_node WHERE application_id={};".format(_CATTLE_TROUGH_MONITOR)
df = pd.read_sql(nodequery, conn)

logger(_LOG_DEBUG, "number of trough nodes {}".format(len(df.index)))
for ind in df.index:
  ticketType = None
  node_id = df['idnode'][ind]
  nodename = df['name'][ind]
  logger(_LOG_INFO, "\nAllTroughs processing for node {}".format(nodename))
  location_id = df['location_id'][ind]
  eventquery = "SELECT dval, eval FROM kanji_eventlog WHERE node_id={} AND sensortype_id=39 AND timestamp > '{}' ORDER BY timestamp desc;".format(node_id, starttime)
  logger(_LOG_DEBUG, eventquery)
  df2 = pd.read_sql(eventquery, conn)
  samplesize = len(df2.index)
  logger(_LOG_DEBUG, "samplesize {}".format(samplesize))
  wmaMean = 0.0
  wmaSd = 0.0
  divisor = 0 
  for ind2 in df2.index:
    logger(_LOG_DEBUG, "{} {}".format(float(df2['dval'][ind2]), float(df2['eval'][ind2])))
    wmaMean += float(df2['dval'][ind2]) * (samplesize - ind2)
    wmaSd   += float(df2['eval'][ind2]) * (samplesize - ind2)
    divisor += (samplesize - ind2)    
  wmaMean = wmaMean/divisor
  wmaSd   = wmaSd/divisor
  logger(_LOG_INFO, "wmaMean={} wmaSd={}".format(wmaMean, wmaSd))
  if (wmaSd<= _MIN_SD) or (wmaMean>_MAX_MEAN) or (wmaMean<_MIN_MEAN):
    logger(_LOG_INFO, "wmaSd below critical value, possible failed sensor.")
    ticketType = BAD_FLUIDLEVEL_SENSOR
    description = "BAD FLUID LEVEL SENSOR. wmaMean={:5.2f} wmaSd={:5.2f}".format(wmaMean, wmaSd)
  elif wmaMean>=_THRESHOLD_WMAMEAN:
    logger(_LOG_INFO, "WATER LEVEL OK. wmaMean={:5.2f} wmaSd={:5.2f}".format(wmaMean, wmaSd))    
  else:
    ticketType = LOW_WATER_LEVEL
    description = "LOW WATER LEVEL. wmaMean={:5.2f} wmaSd={:5.2f}".format(wmaMean, wmaSd)   
  if ticketType is not None:
    #trigger an alert ONLY if the sensor is DRY'    
    locationquery = "SELECT location.idlocation, location.description, location.imageurl, \
                     slackchannel.idslackchannel, slackchannel.channelname, slackchannel.channelid, customer.slacktoken \
                     FROM kanji_location location \
                     JOIN kanji_customer customer ON location.customer_id=customer.idcustomer \
                     JOIN kanji_slackchannel slackchannel ON location.slackalertchannel_id=slackchannel.idslackchannel \
                     WHERE idlocation={}".format(location_id)
    df3 = pd.read_sql(locationquery, conn)
    locationid = df3["idlocation"][0]
    locationdescription = df3["description"][0]
    _SLACK_TOKEN = df3["slacktoken"][0]
    
    _SLACK_CHANNEL_NAME = df3["channelname"][0]
    _SLACK_CHANNEL_ID = df3["channelid"][0]
    _SLACK_CHANNEL_DBID = df3["idslackchannel"][0]
    
    logger(_LOG_DEBUG, "slackChannelName = {}".format(_SLACK_CHANNEL_NAME))
    logger(_LOG_DEBUG, "slackChannelId   = {}".format(_SLACK_CHANNEL_ID))
    
    logger(_LOG_DEBUG, "locationdata")
    logger(_LOG_DEBUG, "locationQuery={}".format(locationquery))    
    mentions = " @Charlie, @Jared"
    #generate and Slack a new ticket ONLY if there is not a currently open ticket for this issue
    openTicket = kt.ticketExists(conn, node_id, ticketType, [kt._OPEN_STATUS, kt._WORKING_STATUS])
    if openTicket is None:
      ticketid = kt.openticket(conn, node_id, locationid, description, 2, 3, ticketType, _SLACK_CHANNEL_DBID)
      ts = kt.slackticket(nodename, locationdescription, description, mentions, 2, 3, locationimageurl, _SLACK_TOKEN, _SLACK_CHANNEL_NAME, ticketid, 0)
      kt.updateTicket(conn, ticketid, ts)  
      logger(_LOG_INFO, "New ticket {} created for this issue.".format(ticketid))
    else:
      logger(_LOG_INFO, "There is an existing ticket #{} for this issue. Created at {}".format(openTicket['idticket'][0], openTicket['opentimestamp'][0]))      
  else:
      logger(_LOG_INFO, "No ticketable events for node {}".format(nodename))
    


AllTroughs processing for node agMote-20002
wmaMean=780.8453661034306 wmaSd=240.88859703021
WATER LEVEL OK. wmaMean=780.85 wmaSd=240.89
No ticketable events for node agMote-20002
